<a href="https://colab.research.google.com/github/Jacob-McM/big-data/blob/main/PC_parts_loading.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:8 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive

In [3]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-30 00:31:25--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.68MB/s    in 0.2s    

2022-11-30 00:31:26 (5.68 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles

url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
PC_reviews_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)

PC_reviews_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [6]:
## Record count
PC_reviews_df.count()

6908554

In [7]:
# Drop null values
PC_na_df = PC_reviews_df.dropna(how='any')
PC_na_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [8]:
PC_na_df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

KEEP:

|review_id|customer_id|product_id|product_parent|review_date|product_title|customer_count|star_rating |helpful_votes|total_votes|vine|
|---|---|---|---|---|---|---|---|---|---|---|


DROP:

|marketplace|product_category|verified_purchace|review_headline|review_body|
|-|-|-|-|-|















In [9]:
database_df = PC_na_df.drop('marketplace','product_category','verified_purchase','review_headline','review_body')
database_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+
|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|          5|            0|          0|   N| 2015-08-31|
|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|          5|           24|         31|   N| 2015-08-31|
|   20329786| RUXJRZCT6953M|B00PML2GQ8|     982036237|AmazonBasics USB ...|          1|            2|          2|   N| 2015-08-31|
|   14215710| R7EO0UO6BPB71|B001NS0OZ4|     576587596|Transcend P8 15-i...|          1|            0|          0|   N| 2015-08-31|
|   38264512|R39NJY2YJ1JFSV|B00AQMTND2|     964759214|Aleratec SATA Dat...|        

In [10]:
database_count = database_df.groupby("customer_id").agg({"customer_id":"count"})
database_count.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   42560427|                11|
|   14991350|                 6|
|    9002099|                 1|
|   35535911|                 3|
|    1117644|                 1|
|   46909180|                 2|
|   48670265|                 2|
|   43626894|                 2|
|   17083191|                 1|
|   20808998|                 1|
|   36728141|                 8|
|   16306618|                 3|
|   13910751|                 1|
|   27377384|                 1|
|   19005540|                 1|
|   51804200|                 1|
|   47108763|                 6|
|   51451778|                 2|
|   44998024|                 2|
|   32753385|                 1|
+-----------+------------------+
only showing top 20 rows



In [11]:
count_join_df = database_df.join(database_count, on="customer_id", how='inner')
count_join_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+------------------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|count(customer_id)|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+------------------+
|      10090|R34CAR0SJ03YEM|B003L5D0OE|     521418880|Mini SATA Cable w...|          2|            0|          0|   N| 2014-03-24|                 1|
|      10168|R3VDP2WDUNGL6U|B004GYQR12|     648613926|Premium Bluetooth...|          3|            0|          0|   N| 2014-01-23|                 2|
|      10168|R24OEGBGJ3KTBW|B00B7KQ4K2|     124918725|64GB Class 10 Mic...|          1|           66|         66|   N| 2013-03-28|                 2|
|      10206|R28AFZSJ1JBII9|B00BHNNQBY|     603357256|Poetic Slimbook L...|          5|            0

In [12]:
database_count_df = count_join_df.withColumnRenamed("count(customer_id)","customer_count")
database_count_df.show()

+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+--------------+
|customer_id|     review_id|product_id|product_parent|       product_title|star_rating|helpful_votes|total_votes|vine|review_date|customer_count|
+-----------+--------------+----------+--------------+--------------------+-----------+-------------+-----------+----+-----------+--------------+
|      10090|R34CAR0SJ03YEM|B003L5D0OE|     521418880|Mini SATA Cable w...|          2|            0|          0|   N| 2014-03-24|             1|
|      10168|R3VDP2WDUNGL6U|B004GYQR12|     648613926|Premium Bluetooth...|          3|            0|          0|   N| 2014-01-23|             2|
|      10168|R24OEGBGJ3KTBW|B00B7KQ4K2|     124918725|64GB Class 10 Mic...|          1|           66|         66|   N| 2013-03-28|             2|
|      10206|R28AFZSJ1JBII9|B00BHNNQBY|     603357256|Poetic Slimbook L...|          5|            0|          0|   N| 2014-

In [14]:
# Load a function that allows us to select columns
from pyspark.sql.functions import col

review_id_df = database_count_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R34CAR0SJ03YEM|      10090|B003L5D0OE|     521418880| 2014-03-24|
|R3TXK1GN22U6UO|      10140|B00K0OBEE2|     644761578| 2015-01-26|
|R3VDP2WDUNGL6U|      10168|B004GYQR12|     648613926| 2014-01-23|
|R24OEGBGJ3KTBW|      10168|B00B7KQ4K2|     124918725| 2013-03-28|
|R28AFZSJ1JBII9|      10206|B00BHNNQBY|     603357256| 2014-03-25|
|R3622VXACD25BM|      10206|B00AMVD9SC|     666314203| 2014-03-25|
|R3BIMY6X5F9YM4|      10206|B00DUGJQEW|     823330197| 2014-03-12|
| R3QLK4D0YMAJ4|      10206|B002WE4HE2|     589324478| 2014-03-12|
| RPGQPG0T2FUJF|      10272|B00IRW1RDW|     236398619| 2015-07-22|
| R7IVQU17B3NXP|      10368|B00JQZJ4OS|     937857628| 2015-06-11|
| R9DZ1P2UNVWCO|      10368|B00H7TVB8C|     120771410| 2014-08-03|
|R2SYKIXT24NQFS|      10368|B008YCYXHC|      13457726| 2014-08

In [16]:
products_df = database_count_df.select(['product_id','product_title'])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B003L5D0OE|Mini SATA Cable w...|
|B004GYQR12|Premium Bluetooth...|
|B00B7KQ4K2|64GB Class 10 Mic...|
|B00BHNNQBY|Poetic Slimbook L...|
|B00AMVD9SC|Insten Universal ...|
|B00DUGJQEW|Samsung Galaxy Ta...|
|B002WE4HE2|Transcend 8 GB Cl...|
|B00IRW1RDW|Fintie Dell Venue...|
|B008X6C4I4|HDE Mini DisplayP...|
|B00951ULKO|INVELLOP CRYSTAL ...|
|B005AOKW8Q|Generic Styli for...|
|B00GU8OIYA|Sabrent USB 2.0 H...|
|B004EGEMMI|          SATA Cable|
|B003M75D7I|Molex 4 Pin to 2 ...|
|B0033Z2BAQ|Asus 24x DVD-RW S...|
|B003IYS8SQ|Tenext Cat5 Netwo...|
|B004S7WTJQ|Laptone 150Mbps H...|
|B00N02BAV8|LG GH24NSC0B DVD ...|
|B00BTQJ7SA|Rii I8 Mini 2.4Gh...|
|B0031WNEHU|14.80V,2600mAh,Li...|
+----------+--------------------+
only showing top 20 rows



In [15]:
customer_df = database_count_df.select(['customer_id','customer_count'])
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|      10016|             1|
|      10019|             2|
|      10019|             2|
|      10040|             1|
|      10045|             1|
|      10052|             1|
|      10090|             1|
|      10136|             1|
|      10140|             1|
|      10141|             4|
|      10141|             4|
|      10141|             4|
|      10141|             4|
|      10147|             1|
|      10168|             2|
|      10168|             2|
|      10206|             4|
|      10206|             4|
|      10206|             4|
|      10206|             4|
+-----------+--------------+
only showing top 20 rows



In [18]:
vine_df = database_count_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RUJTTWQWR0SVF|          5|            0|          0|   N|
|R1Q7GL6UB7DX50|          5|            9|         11|   N|
|R1X31YNZLSEI32|          5|            0|          0|   N|
| RJ7QGUJT34AWZ|          5|            1|          2|   N|
|R34CAR0SJ03YEM|          2|            0|          0|   N|
|R1SN57A6HZZEI6|          4|            0|          0|   N|
|R3TXK1GN22U6UO|          5|            0|          0|   N|
|R3VDP2WDUNGL6U|          3|            0|          0|   N|
|R24OEGBGJ3KTBW|          1|           66|         66|   N|
|R28AFZSJ1JBII9|          5|            0|          0|   N|
|R3622VXACD25BM|          5|            0|          0|   N|
|R3BIMY6X5F9YM4|          4|            0|          0|   N|
| R3QLK4D0YMAJ4|          2|            0|          1|   N|
| RPGQPG0T2FUJF|          3|            

Postgres Setup

In [19]:
## Values hidden for privacy

aws_endpoint = '###'
aws_port = '###'
aws_db = '###'
aws_username = '###'
aws_password = '###'

In [20]:
jdbc_url=f'jdbc:postgresql://{aws_endpoint}:{aws_port}/{aws_db}'

config = {"user": f'{aws_username}', 
          "password": f'{aws_password}', 
          "driver":"org.postgresql.Driver"}

mode = 'overwrite' 

In [21]:
review_id_df .write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_df .write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [23]:
customer_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [24]:
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)